In [1]:
from nltk.tokenize import word_tokenize
import nltk
import scipy as sp

1. term frequency(tf)
    - 어떤 단어가 문서 내에서 자주 등장할수록, 중요도가 높아진다
2. inverse document frequency(idf)
    - 비교하는 모든 문서 내에서 만약에 같은 단어가 존재한다면, 핵심 어휘일 수도 있다 
    - 그러나, 문서 간의 비교에서는 중요한 단어가 아닐 수 있다  

- 텍스트 마이닝에서 사용하는 기법
- 단어별로 가중치를 부과하는 방식

In [11]:
for w in set(['a', 'b', 'b']) :  # {a, b}
    print(['a', 'b', 'b'].count(w))   # a는 1개, b는 2개

1
2


In [12]:
# d.count(w) for w in set(d)
# 문장 안의 단어별 빈도수
print([['a'].count(w) for w in set(['a'])])
print([['a', 'b', 'b'].count(w) for w in set(['a', 'b', 'b'])])
print([['a', 'b', 'c'].count(w) for w in set(['a', 'b', 'c'])])

[1]
[1, 2]
[1, 1, 1]


In [17]:
a, abb, abc = ['a'], ['a', 'b', 'b'], ['a', 'b', 'c']
# 여러 문장을 모아 둔 리스트
D = [a, abb, abc]
len(D)

3

In [23]:
for doc in D :
    if 'b' in doc :
        print(doc)

['a', 'b', 'b']
['a', 'b', 'c']


In [21]:
# 문장 리스트 내에서 해당 단어가 포함된 문장의 개수
len([ doc for doc in D if 'b' in doc])

2

In [26]:
# 함수 구성
'''
t : 단어, 'a' or 'b' or 'c'
d : 문장을 가진 한 세트, ['a', 'b', 'b']
D : 문장 리스트를 멤버로 가진 리스트(비교할 문장 덩어리), 
    [['a'], ['a', 'b', 'b'], ['a', 'b', 'c']]
'''
def tfidf(t, d, D) :
    # tf : 문장 전체의 단어 빈도수 대비 해당 단어의 빈도수 비율
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    
    idf = sp.log(float(len(D)) / len([ doc for doc in D if 'b' in doc]))
    return tf, idf

In [28]:
# 단어 빈도수가 1인데 'a'도 1번 등장했으므로 tf = 1
print(tfidf('a', a, D))
# 단어 빈도수가 3인데 'b'는 2번 등장했으므로 tf = 0.67
print(tfidf('b', abb, D))
print(tfidf('a', abc, D))
print(tfidf('b', abc, D))
print(tfidf('c', abc, D))

(1.0, 0.4054651081081644)
(0.6666666666666666, 0.4054651081081644)
(0.3333333333333333, 0.4054651081081644)
(0.3333333333333333, 0.4054651081081644)
(0.3333333333333333, 0.4054651081081644)


###### vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

In [29]:
# 라이브러리로 처리
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

In [33]:
# 형태소 분석기
from konlpy.tag import Okt
t = Okt()

In [30]:
contents = [
    '길동이랑 술마시고 싶지만 바쁜데 어떡하죠?', 
    '길동이는 공원에서 산책하고 노는 것을 싫어해요', 
    '길동이는 공원에서 노는 것도 싫어해요. 이상해요.', 
    '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요.'
]

In [35]:
for doc in contents :
    # print(doc)
    print(t.morphs(doc))

['길동', '이랑', '술', '마시고', '싶지만', '바쁜데', '어떡하죠', '?']
['길동', '이', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요']
['길동', '이', '는', '공원', '에서', '노', '는', '것', '도', '싫어해요', '.', '이상해요', '.']
['먼', '곳', '으로', '여행', '을', '떠나고', '싶은데', '너무', '바빠서', '그러질', '못', '하고', '있어요', '.']


In [36]:
contents_tokens = [ t.morphs(doc) for doc in contents ]
contents_tokens

[['길동', '이랑', '술', '마시고', '싶지만', '바쁜데', '어떡하죠', '?'],
 ['길동', '이', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요'],
 ['길동', '이', '는', '공원', '에서', '노', '는', '것', '도', '싫어해요', '.', '이상해요', '.'],
 ['먼',
  '곳',
  '으로',
  '여행',
  '을',
  '떠나고',
  '싶은데',
  '너무',
  '바빠서',
  '그러질',
  '못',
  '하고',
  '있어요',
  '.']]

In [37]:
len(contents_tokens), len(contents_tokens[0])

(4, 8)

In [40]:
# 형태소 기반으로 문장을 재구성
# '길동', '이랑', '술', '마시고', '싶지만', '바쁜데', '어떡하죠', '?'
# => 길동 이랑 술 마시고 싶지만 바쁜데 어떡하죠 ?
contents_for_vectorize = []
for content in contents_tokens :
    print(content)
    tmp = ''
    for word in content :
        tmp = tmp + ' ' + word
    contents_for_vectorize.append(tmp)

['길동', '이랑', '술', '마시고', '싶지만', '바쁜데', '어떡하죠', '?']
['길동', '이', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요']
['길동', '이', '는', '공원', '에서', '노', '는', '것', '도', '싫어해요', '.', '이상해요', '.']
['먼', '곳', '으로', '여행', '을', '떠나고', '싶은데', '너무', '바빠서', '그러질', '못', '하고', '있어요', '.']


In [41]:
contents_for_vectorize

[' 길동 이랑 술 마시고 싶지만 바쁜데 어떡하죠 ?',
 ' 길동 이 는 공원 에서 산책 하고 노 는 것 을 싫어해요',
 ' 길동 이 는 공원 에서 노 는 것 도 싫어해요 . 이상해요 .',
 ' 먼 곳 으로 여행 을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요 .']

In [42]:
# 다른 방법
contents_for_vectorize = []
contents_for_vectorize = [' '.join(content) for content in contents_tokens ]

In [43]:
contents_for_vectorize

['길동 이랑 술 마시고 싶지만 바쁜데 어떡하죠 ?',
 '길동 이 는 공원 에서 산책 하고 노 는 것 을 싫어해요',
 '길동 이 는 공원 에서 노 는 것 도 싫어해요 . 이상해요 .',
 '먼 곳 으로 여행 을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요 .']

###### 벡터화 : vectorizer.fit_transform( )

In [45]:
# 벡터화
X = vectorizer.fit_transform(contents_for_vectorize)
X.shape

(4, 20)

In [46]:
# 데이터의 크기를 변수로 받는다
num_samples, num_features = X.shape
num_samples, num_features

(4, 20)

In [49]:
# 벡터의 특성화값 -> 컬럼 -> 변수, 특성
vectorizer.get_feature_names()

['공원',
 '그러질',
 '길동',
 '너무',
 '떠나고',
 '마시고',
 '바빠서',
 '바쁜데',
 '산책',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떡하죠',
 '에서',
 '여행',
 '으로',
 '이랑',
 '이상해요',
 '있어요',
 '하고']

In [50]:
# 신규 문장 -> 벡터화
# 1. 신문장
new_post = ['길동이랑 공원에서 산책하고 놀고 싶어요']
# 2. 형태소 분리
new_post_tokens = [t.morphs(doc) for doc in new_post]
new_post_tokens

[['길동', '이랑', '공원', '에서', '산책', '하고', '놀고', '싶어요']]

In [51]:
# 3. 형태소 단위로 문장 재구성
new_post_for_vectorize = [' '.join(content) for content in new_post_tokens]
new_post_for_vectorize

['길동 이랑 공원 에서 산책 하고 놀고 싶어요']

In [52]:
# 벡터화
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.shape

(1, 20)

In [70]:
# 정규화 적용하여 거리 계산(벡터화)
def dist_norm(v1, v2) :
    # 각각 벡터 문장의 정규화
    v1_norm = v1 / sp.linalg.norm(v1.toarray())
    v2_norm = v2 / sp.linalg.norm(v2.toarray())
    # 두 문장의 거리(차이) 계산
    delta = v1_norm - v2_norm
    # 결과 리턴
    return sp.linalg.norm(delta.toarray())

In [71]:
# 유사도 검사
best_doc = None
best_distance = 1000
best_index = None
for i in range(num_samples) : # num_samples : 데이터 수
    # 비교 문장의 벡터값 획득
    post_vec = X.getrow(i)
    # print(post_vec)
    # 거리 계산
    d = dist_norm(post_vec, new_post_vec)
    # 기준값보다 작을 때 처리
    if d < best_distance :
        # 거리값
        best_distance = d
        # 유사한 인덱스
        best_index = i
        # 유사한 문장
        best_doc = contents[best_index]

In [72]:
# 결과 확인
best_distance

0.6288361746517446

In [73]:
best_index

1

In [74]:
best_doc

'길동이는 공원에서 산책하고 노는 것을 싫어해요'